In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import torch.hub 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [2]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=False)
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Linear(num_features, 3 )

Using cache found in C:\Users\nisha/.cache\torch\hub\pytorch_vision_v0.10.0
c:\users\nisha\appdata\local\programs\python\python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\users\nisha\appdata\local\programs\python\python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [14]:
dataset = datasets.ImageFolder(r"C:\Users\nisha\COMP6721\signs_train\asl_alphabet_train\asl_alphabet_train", transform=preprocess)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [15]:
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [16]:
len(dataset.classes)

3

In [19]:
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
model.to(device)

num_epochs = 5
total_steps = len(train_dataloader)

t1 = time.time()
loss
for epoch in range(num_epochs):
    for i, data in enumerate(train_dataloader):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/10], Step [50/141], Loss: 0.2010, Accuracy: 90.62%
Epoch [1/10], Step [100/141], Loss: 0.0398, Accuracy: 100.00%
Epoch [2/10], Step [50/141], Loss: 0.2303, Accuracy: 96.88%
Epoch [2/10], Step [100/141], Loss: 0.0020, Accuracy: 100.00%
Epoch [3/10], Step [50/141], Loss: 0.0002, Accuracy: 100.00%
Epoch [3/10], Step [100/141], Loss: 0.0006, Accuracy: 100.00%
Epoch [4/10], Step [50/141], Loss: 0.0000, Accuracy: 100.00%
Epoch [4/10], Step [100/141], Loss: 0.0001, Accuracy: 100.00%
Epoch [5/10], Step [50/141], Loss: 0.0001, Accuracy: 100.00%
Epoch [5/10], Step [100/141], Loss: 0.0000, Accuracy: 100.00%
Epoch [6/10], Step [50/141], Loss: 0.0000, Accuracy: 100.00%
Epoch [6/10], Step [100/141], Loss: 0.0000, Accuracy: 100.00%
Epoch [7/10], Step [50/141], Loss: 0.0000, Accuracy: 100.00%
Epoch [7/10], Step [100/141], Loss: 0.0000, Accuracy: 100.00%
Epoch [8/10], Step [50/141], Loss: 0.0000, Accuracy: 100.00%
Epoch [8/10], Step [100/141], Loss: 0.0000, Accuracy: 100.00%
Epo

In [22]:
model.eval() 

with torch.no_grad(): 
    correct = 0
    total = 0
    for data in test_dataloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))

Test Accuracy of the model on the 9000 test images: 100.0 %
